In [ ]:
import os
import napari
import numpy as np
import tifffile
import pathlib
import re
from skimage import io, util
from scipy import ndimage
import pyautogui
import time

from enum import Enum
from napari.utils.colormaps import Colormap

# All types colorblind-friendly colors palette
class ColorblindFriendlyColors(Enum):
    PurpleBlind = [129, 50, 130, 255] # colors are not optimized for colorblind
    CyanBlind = [87, 181, 178, 255] # colors are not optimized for colorblind
    YellowBlind = [253, 225, 81, 255] # colors are not optimized for colorblind
    RedBlind = [149, 34, 25, 255] # colors are not optimized for colorblind

# Create colormap objects
purple_blind_inclusive_colormap = Colormap([[0, 0, 0, 0], ColorblindFriendlyColors.PurpleBlind.value])
cyan_blind_inclusive_colormap = Colormap([[0, 0, 0, 0], ColorblindFriendlyColors.CyanBlind.value])
yellow_blind_inclusive_colormap = Colormap([[0, 0, 0, 0], ColorblindFriendlyColors.YellowBlind.value])
red_blind_inclusive_colormap = Colormap([[0, 0, 0, 0], ColorblindFriendlyColors.RedBlind.value])

def Inverted_MAX_Projection_Single_Cell(path):

    #os.chdir(fr'\\?\D:\Projects\Calm_ISO\Vlad_Dataset_Organized)

    with open(os.path.join(path, fr'' + os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()

    pattern = r"XYZCalibration\(um\/vox\):(\d+\.\d+),(\d+\.\d+),(\d+\.\d+):"
    match = re.search(pattern, content)

    if match:
        x_dim = float(match.group(1))
        y_dim = float(match.group(2))
        z_dim = float(match.group(3))

    # Extract channel numbers and names using regular expressions
    # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+)', content)[0]))
    # channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+)', content)[0]

    channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+),(\d+)', content)[0])) #ONLY for 3rd channel
    channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+),(\w+)', content)[0] #ONLY for 3rd channel      

    # Combine channel numbers and names into a dictionary
    channel_info = dict(zip(channel_numbers, channel_names))

    mask0 = channel_names[0]
    raw0 = channel_numbers[0]
    mask1 = channel_names[1]
    raw1 = channel_numbers[1]
    mask2 = channel_names[2]
    raw2 = channel_numbers[2]
    mask3 = channel_names[3]
    raw3 = channel_numbers[3]

    target1_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask1}_filtered_image.tif')
    target2_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask2}_filtered_image.tif')
    target3_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask3}_filtered_image.tif')
    
    target1_raw_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw1}.tif')
    target2_raw_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw2}.tif')
    target3_raw_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw3}.tif')        
    nuclei_raw_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw0}.tif')  
        
    target1_labeled_path = os.path.join(path, 'Segmented', f'Labeled_{mask1}.tif')
    target2_labeled_path = os.path.join(path, 'Segmented', f'Labeled_{mask2}.tif')
    target3_labeled_path = os.path.join(path, 'Segmented', f'Labeled_{mask3}.tif')
    
    body_mask_path = os.path.join(path, 'Segmented', 'Masked_Body_image.tif')
    cytosol_mask_path = os.path.join(path, 'Segmented', 'Masked_Body_image.tif')    
    nuclei_mask_path = os.path.join(path, 'Segmented', 'Masked_Nuclei_image.tif')     

    target_mask1 = io.imread(target1_mask_path)
    target_mask2 = io.imread(target2_mask_path)
    target_mask3 = io.imread(target3_mask_path)
    
    target_raw1 = io.imread(target1_raw_path)
    target_raw2 = io.imread(target2_raw_path)
    target_raw3 = io.imread(target3_raw_path)       
    nuclei_raw = io.imread(nuclei_raw_path)       
    
#     target1_labeled = io.imread(target1_labeled_path)    
#     target2_labeled = io.imread(target2_labeled_path)
#     target3_labeled = io.imread(target3_labeled_path)

    body_mask = io.imread(body_mask_path)
    cytosol = io.imread(cytosol_mask_path)
    nuclei_mask = io.imread(nuclei_mask_path)
    
    # overlap12 = target_mask1*target_mask2
    # overlap23 = target_mask2*target_mask3
    # overlap13 = target_mask3*target_mask1
    overlap123 = target_mask3*target_mask2*target_mask1


    #this code first removes and then adds layers of the top and bottom of the body mask TIFF
    body_shape = body_mask.shape[0]
    body_middle =  body_mask[1:body_shape-1]
    body_BOTTOM_contour = util.img_as_bool(np.zeros((1, body_middle.shape[1], body_middle.shape[2])))
    body_BOTTOM = util.img_as_uint(ndimage.binary_fill_holes(body_BOTTOM_contour))
    body_TOP_contour = util.img_as_bool(np.zeros((1, body_middle.shape[1], body_middle.shape[2])))
    body_TOP = util.img_as_uint(ndimage.binary_fill_holes(body_TOP_contour))

    body = np.vstack((body_BOTTOM,body_middle,body_TOP)) 

    white_layer = util.img_as_ubyte(np.zeros((body_shape, body_middle.shape[1], body_middle.shape[2])))
    background = util.img_as_uint(np.invert(white_layer))

    viewer = napari.view_image(background, gamma=2, iso_threshold = 0, rendering='iso', depiction='volume', blending='additive', colormap='gray', name='background', scale=(z_dim, y_dim, x_dim))

    viewer.add_image(target_mask1, gamma=1, rendering='attenuated_mip',depiction='volume', blending='translucent', colormap= red_blind_inclusive_colormap, name=f'{mask1}', scale=(z_dim, y_dim, x_dim))
    viewer.add_image(target_mask2, gamma=1, rendering='attenuated_mip',depiction='volume', blending='translucent', colormap= cyan_blind_inclusive_colormap , name=f'{mask2}', scale=(z_dim, y_dim, x_dim))
    viewer.add_image(target_mask3, gamma=1, rendering='attenuated_mip',depiction='volume', blending='translucent', colormap= purple_blind_inclusive_colormap , name=f'{mask3}', scale=(z_dim, y_dim, x_dim))
    viewer.add_image(nuclei_mask, gamma=1, interpolation3d  = 'linear', iso_threshold = 0, rendering='attenuated_mip', depiction='volume', opacity=1, blending='translucent', colormap='gray', name='nuclei', scale=(z_dim, y_dim, x_dim))

    viewer.add_image(overlap123, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))

    # viewer.add_image(overlap12, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))
    # viewer.add_image(overlap13, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))#     viewer.add_image(overlap12, gamma=0.1, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent', colormap='yellow', name=f'{mask1}x{mask2}', scale=(z_dim, y_dim, x_dim))
    # viewer.add_image(overlap23, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))

    viewer.add_image(target_raw1, gamma=1, rendering='additive',depiction='volume', blending='additive', colormap= red_blind_inclusive_colormap, name=f'{mask1}', scale=(z_dim, y_dim, x_dim))
    viewer.add_image(target_raw2, gamma=1, rendering='additive',depiction='volume', blending='additive', colormap= cyan_blind_inclusive_colormap, name=f'{mask2}', scale=(z_dim, y_dim, x_dim))
    viewer.add_image(target_raw3, gamma=1, rendering='additive',depiction='volume', blending='additive', colormap= purple_blind_inclusive_colormap, name=f'{mask3}', scale=(z_dim, y_dim, x_dim))
    viewer.add_image(nuclei_raw, gamma=2, interpolation3d  = 'linear', rendering='additive', depiction='volume', opacity=1, blending='additive', colormap='gray', name='nuclei', scale=(z_dim, y_dim, x_dim))

#     viewer.add_labels(target1_labeled, name=f'{mask1}_labeled', num_colors=np.max(target1_labeled) + 1, scale=(z_dim, y_dim, x_dim))
#     viewer.add_labels(target2_labeled, name=f'{mask2}_labeled', num_colors=np.max(target2_labeled) + 1, scale=(z_dim, y_dim, x_dim))
#     viewer.add_labels(target3_labeled, name=f'{mask3}_labeled', num_colors=np.max(target3_labeled) + 1, scale=(z_dim, y_dim, x_dim))

    viewer.add_image(body, gamma=0.7, interpolation3d  = 'linear', iso_threshold = 0, rendering='iso', depiction='volume', blending='minimum', opacity=0.10, colormap='gray', name='body', scale=(z_dim, y_dim, x_dim))


#     Create a scale bar with a specified size and color
    viewer.scale_bar.visible = True
    viewer.scale_bar.color = "black"
#     viewer.scale_bar.font_size = 47
#     viewer.scale_bar.colored = True
    viewer.scale_bar.unit = "um"
    viewer.theme = 'dark' #or "light"
    viewer.dims.ndisplay = 3
    viewer.camera.zoom = 20
       
    viewer.text_overlay.text = f'{mask1}: {red_blind_inclusive_colormap}\n {mask2}: {cyan_blind_inclusive_colormap}\n {mask3}: {purple_blind_inclusive_colormap}\n Overlap: gray'
    viewer.text_overlay.color = 'black' # or  (0.5, 0.5, 0.5, 1.0)
    viewer.text_overlay.font_size = 27
    viewer.text_overlay.position = "bottom_center" # "top_left", "top_right", "top_center", "bottom_right", "bottom_left"
    viewer.text_overlay.visible = True
    viewer.window.resize(3840, 2160)

In [ ]:
path = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3\2023-09-26_C57BL6_M_Saline-4h_Cacna1c-488_NCad-568_Calm3-647_c1_deconv'
Inverted_MAX_Projection_Single_Cell(path)